In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()

print("imports 1 complete")

imports 1 complete


In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = "C:\\Users\\rahul\\Documents\\work\\BuildUCLA\\data\\printed_with_ids\\images"

set_types = ['train', 'val', 'test']

# Need to split data sets into training and testing sets
data_sets = {t : datasets.ImageFolder(data_dir, transform=data_transforms[t])
             for t in set_types}

# Need to split data into training set, validation set, and testing set
train_size = 0.65     # 65% of all data is training
val_size = 0.15       # 15% of all data is validation
test_size = (1 - train_size - val_size)    # Remaining data (20%) is testing

num_images = len(data_sets["train"]) # length of both sets should be the same
all_ind = list(range(num_images))
random_seed = 11
np.random.seed(random_seed)
np.random.shuffle(all_ind)

train_split = int(num_images * train_size)
val_split = int(num_images * val_size)
test_split = int(num_images * test_size)

data_samplers = {}
data_samplers["train"] = all_ind[:train_split]
data_samplers["val"] = all_ind[train_split : train_split+val_split]
data_samplers["test"] = all_ind[train_split+val_split:]

dataloaders = {t : torch.utils.data.DataLoader(data_sets[t],
                                              sampler=data_samplers[t],
                                              batch_size=4,
                                              num_workers=4)
              for t in set_types}

dataset_sizes = {t : len(data_samplers[t]) for t in set_types}

class_names = data_sets["train"].classes
use_gpu = torch.cuda.is_available()

print("data loaded from " + str(data_dir))
print("read classes: " + str(class_names))
if use_gpu:
    print("use_gpu is true")
else:
    print("use_gpu is false")

data loaded from C:\Users\rahul\Documents\work\BuildUCLA\data\printed_with_ids\images
read classes: ['negative', 'positive']
use_gpu is true


In [3]:
for set_type in set_types:
    poscount = 0
    totcount = 0
    for inputs, labels in dataloaders[set_type]:
        for label in labels:
            totcount = (totcount + 1)
            if label == 1:
                poscount = (poscount + 1)
    print("set_type = " + str(set_type))
    print("total = " + str(totcount))
    print("pos = " + str(poscount))
    print("pos ratio = " + str(float(poscount)/float(totcount)))
    print("")

set_type = train
total = 1832
pos = 1290
pos ratio = 0.7041484716157205

set_type = val
total = 422
pos = 299
pos ratio = 0.7085308056872038

set_type = test
total = 565
pos = 389
pos ratio = 0.6884955752212389



In [6]:
def load_model(weights_path):
    model_resnet18 = models.resnet18(pretrained=False)
    num_ftrs = model_resnet18.fc.in_features
    model_resnet18.fc = nn.Linear(num_ftrs, 2)
    print("acquired resnet18 architecture")

    # weights_path = "resnet18_half_frozen_5epochs_transfer-state.pt"
    model_resnet18.load_state_dict(torch.load(weights_path))
    model_resnet18.cuda()
    print("loaded weights from " + weights_path)
    
    return model_resnet18

In [8]:
print("try to evaluate model")

import torchnet as tnt

for epoch in range(10):
    model_resnet18 = load_model("resnet18_half_frozen_" + str(epoch + 1) + "epochs_transfer-state.pt")
    model_resnet18.eval()
    confusion_matrix = tnt.meter.ConfusionMeter(2)

    for i, data in enumerate(dataloaders["test"]):
        input, label = data
        input_var = Variable(input, volatile=True).cuda()
        label_var = Variable(label, volatile=True).cuda()
        score = model_resnet18(input_var)
        confusion_matrix.add(score.data,
                             label_var.data)

    print("Epoch " + str(epoch + 1))
    print("printing confusion matrix of format:")
    print("[[TN FP]\n [FN TP]]")
    print(confusion_matrix.conf)
    print()
print("evaluation complete")

try to evaluate model
acquired resnet18 architecture
loaded weights from resnet18_half_frozen_1epochs_transfer-state.pt
Epoch 1
printing confusion matrix of format:
[[TN FP]
 [FN TP]]
[[112  64]
 [ 65 324]]

acquired resnet18 architecture
loaded weights from resnet18_half_frozen_2epochs_transfer-state.pt
Epoch 2
printing confusion matrix of format:
[[TN FP]
 [FN TP]]
[[141  35]
 [100 289]]

acquired resnet18 architecture
loaded weights from resnet18_half_frozen_3epochs_transfer-state.pt
Epoch 3
printing confusion matrix of format:
[[TN FP]
 [FN TP]]
[[114  62]
 [ 55 334]]

acquired resnet18 architecture
loaded weights from resnet18_half_frozen_4epochs_transfer-state.pt
Epoch 4
printing confusion matrix of format:
[[TN FP]
 [FN TP]]
[[130  46]
 [ 57 332]]

acquired resnet18 architecture
loaded weights from resnet18_half_frozen_5epochs_transfer-state.pt
Epoch 5
printing confusion matrix of format:
[[TN FP]
 [FN TP]]
[[120  56]
 [ 43 346]]

acquired resnet18 architecture
loaded weights fr